# Sora2 Watermark Remover (Local Jupyter Version)

This notebook is a tool for removing watermarks from Sora videos, designed for local Jupyter environments.

**New Features:**
- `--frame-step`: Frame processing interval (1=all frames, 2=every other frame...)
- `--target-fps`: Output fps specification (0=same as input)

**Repository:** https://github.com/fulfulggg/Sora2WatermarkRemover

## 1. Setup

### ⚠️ Important: Pillow Dependency Fix

**Run the cell below, then restart the kernel.**

1. Run the cell below
2. Select "Kernel" → "Restart Kernel"
3. After restart, continue execution from the next cell

In [ ]:
# Fix Pillow dependencies (restart kernel after running this)
!pip uninstall -y pillow PIL
!pip install -U "pillow==10.4.0"

# Optional: suppress websockets warnings
!pip install -U "websockets==15.0.1"

print("\n✅ Done. Please run: Kernel → Restart Kernel")

In [ ]:
# Verify after restart (run this cell first)
import PIL
import PIL._util as u

print(f"Pillow version: {PIL.__version__}")  # => 10.4.0
print(f"is_directory exists: {hasattr(u, 'is_directory')}")  # => True

if PIL.__version__ == "10.4.0" and hasattr(u, "is_directory"):
    print("✅ Pillow dependencies are OK. Proceed to the next cell.")
else:
    print("⚠️ There's a problem. Did you restart the kernel?")

In [ ]:
# Install required packages
!pip install -q transformers torch opencv-python tqdm loguru iopaint ipywidgets

print("✅ Package installation completed.")
print("\n📝 Note: ffmpeg is required. If not installed, run the following command:")
print("   Ubuntu/Debian: sudo apt-get install -y ffmpeg")
print("   macOS: brew install ffmpeg")
print("   Windows: Download from https://ffmpeg.org/download.html")

## 2. Device Check

In [ ]:
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

if torch.cuda.is_available():
    print(f"CUDA device: {torch.cuda.get_device_name(0)}")
    print(f"CUDA version: {torch.version.cuda}")
else:
    print("⚠️ GPU not available. Processing with CPU (may take longer).")

## 3. Input Video Configuration

You have two options to specify the input video:

**Option A: Upload a file** (recommended for ease of use)
- Run the next cell and use the upload button

**Option B: Specify a file path manually**
- Edit the `input_video` variable in the next cell with your file path

In [ ]:
import os
import ipywidgets as widgets
from IPython.display import display, clear_output

# Global variable to store the input video path
input_video = None

# Create upload widget
upload_widget = widgets.FileUpload(
    accept='video/*,.mp4,.avi,.mov,.mkv',
    multiple=False,
    description='Upload Video'
)

# Create manual path input widget
path_input = widgets.Text(
    value='',
    placeholder='Or enter file path here (e.g., /path/to/video.mp4)',
    description='File Path:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='600px')
)

# Output widget for status messages
output = widgets.Output()

def on_upload_change(change):
    """Handle file upload"""
    with output:
        clear_output()
        if upload_widget.value:
            # Get the uploaded file
            uploaded_file = list(upload_widget.value.values())[0]
            filename = list(upload_widget.value.keys())[0]
            
            # Save to current directory
            global input_video
            input_video = os.path.abspath(filename)
            
            with open(input_video, 'wb') as f:
                f.write(uploaded_file['content'])
            
            file_size = os.path.getsize(input_video) / (1024 * 1024)  # MB
            print(f"✅ File uploaded successfully: {input_video}")
            print(f"📊 File size: {file_size:.2f} MB")

def on_path_change(change):
    """Handle manual path input"""
    with output:
        clear_output()
        if path_input.value.strip():
            global input_video
            input_video = path_input.value.strip()
            
            if os.path.exists(input_video):
                abs_path = os.path.abspath(input_video)
                file_size = os.path.getsize(abs_path) / (1024 * 1024)  # MB
                input_video = abs_path
                print(f"✅ Input file verified: {abs_path}")
                print(f"📊 File size: {file_size:.2f} MB")
            else:
                print(f"❌ Error: Input file not found: {input_video}")
                print("\n💡 Hint: Specify using one of the following methods:")
                print("   1. Absolute path: /full/path/to/video.mp4")
                print("   2. Relative path: ./videos/input.mp4")
                print("   3. Same directory as notebook: video.mp4")

# Attach event handlers
upload_widget.observe(on_upload_change, names='value')
path_input.observe(on_path_change, names='value')

# Display widgets
print("📤 Option A: Upload a video file")
display(upload_widget)
print("\n📝 Option B: Enter file path manually")
display(path_input)
print("\n" + "="*60)
display(output)

### Verify Input File

Run the cell below to verify your input file is set correctly before processing.

In [ ]:
# Verify the input file is set
if input_video and os.path.exists(input_video):
    file_size = os.path.getsize(input_video) / (1024 * 1024)  # MB
    print(f"✅ Ready to process:")
    print(f"   File: {input_video}")
    print(f"   Size: {file_size:.2f} MB")
elif input_video:
    print(f"❌ Error: File path set but file not found: {input_video}")
else:
    print("⚠️ No input file selected yet. Please upload a file or specify a path in the cell above.")

## 4. Video Processing

### Parameter Configuration

- `max_bbox_percent`: Maximum bounding box size (percentage of image)
- `frame_step`: Frame processing interval (1=all frames, 2=every other frame...)
- `target_fps`: Output fps (0=same as input)

In [ ]:
import os

# Parameter configuration (adjust as needed)
max_bbox_percent = 10.0  # Maximum bounding box size
frame_step = 1           # 1=all frames, 2=every other frame
target_fps = 0.0         # 0=same as input

# Output file name configuration
output_video = "output_no_watermark.mp4"

# Verify input file
if not os.path.exists(input_video):
    print(f"❌ Error: Input file not found: {input_video}")
    print("Please set the input file path correctly in the previous cell.")
else:
    input_path = os.path.abspath(input_video)
    output_path = os.path.abspath(output_video)
    
    print(f"✅ Input file: {input_path}")
    print(f"📝 Output destination: {output_path}")
    print(f"⚙️ Parameters: max_bbox_percent={max_bbox_percent}, frame_step={frame_step}, target_fps={target_fps}")
    print("\n🚀 Starting processing...\n")
    
    # Execute remwm.py
    !python remwm.py "{input_path}" "{output_path}" \
      --max-bbox-percent {max_bbox_percent} \
      --frame-step {frame_step} \
      --target-fps {target_fps}
    
    # Verify output file
    if os.path.exists(output_path):
        file_size = os.path.getsize(output_path) / (1024 * 1024)  # MB
        print(f"\n✅ Processing completed: {output_path}")
        print(f"📊 File size: {file_size:.2f} MB")
    else:
        print(f"\n❌ Error: Output file was not created")
        print("Please check the error messages above")

## 5. Review Results

The processed video is saved at the specified output path.

In [ ]:
# Display output file information
if os.path.exists(output_video):
    abs_path = os.path.abspath(output_video)
    file_size = os.path.getsize(abs_path) / (1024 * 1024)  # MB
    print(f"📁 Output file: {abs_path}")
    print(f"📊 File size: {file_size:.2f} MB")
    
    # Display with video player (if supported in Jupyter environment)
    try:
        from IPython.display import Video
        print("\n📺 Preview:")
        display(Video(abs_path, width=640))
    except:
        print("\n💡 Please open the file with a video player to review.")
else:
    print("❌ Output file not found. Please verify that processing completed successfully.")

## 6. Usage Tips

### To speed up processing

```python
frame_step = 2  # Process every other frame (approximately 2x faster)
```

### To set a fixed output fps

```python
target_fps = 30  # Output at 30fps
```

### Combination example

```python
frame_step = 2
target_fps = 30
# → Process every other frame and output at 30fps
```

### Batch Processing

To process multiple videos:

```python
import glob

# Process all mp4 files in a specific directory
video_files = glob.glob("./videos/*.mp4")

for video in video_files:
    input_video = video
    output_video = video.replace(".mp4", "_no_watermark.mp4")
    # Run processing code...
```